In [1]:
# Imports

import pandas as pd
import numpy as np
import time
import plotly.express as px
import random


In [2]:
# Eight Queens State

class EightQueensState:
    def __init__(self, board_size, state=None):
        if state is None:
            # Inital board set up
            self.state = np.random.randint(0, board_size, board_size)
        else:
            self.state = np.array(state) 

        # Set the goal fitness value where there is no queen attacks 
        self.goal_fitness = int(((board_size ** 2) - board_size) / 2)
    
    
    def fitness(self):
        count = 0
        n = len(self.state)
        
        for i in range(n - 1):
            remaining_queens = self.state[i + 1:]
            current_queen = self.state[i]
            
            # Count of the queen attacks on the row 
            count += (current_queen == remaining_queens).sum()
            
            # Count of the queen attacks on the diagonal 
            distances = np.arange(1, n - i)
            upper_diagonal = current_queen + distances
            lower_diagonal = current_queen - distances
            
            # Total queen on queen attacks
            count += sum((remaining_queens == upper_diagonal) | (remaining_queens == lower_diagonal))

        # Return the final fitness 
        return self.goal_fitness - count

In [48]:
# Plot GA

def plot_ga(fitness_history):

    # Create DataFrame
    df = pd.DataFrame({
        'Generation': range(len(fitness_history)),
        'Fitness': fitness_history
    })

    # Create the figure with dark theme using plotly express
    fig = px.line(
        df,
        x='Generation',
        y='Fitness',
        title='Eight Queens - Genetic Algorithm Fitness Evolution',
        labels={'Fitness': 'Fitness Score (28 = Perfect Solution)', 'Generation': 'Generation'},
        template='plotly_dark'
    )

    # Add the solution point as a marker if found
    solution_point = df[df['Fitness'] == 28]
    if not solution_point.empty:
        fig.add_scatter(
            x=solution_point['Generation'],
            y=solution_point['Fitness'],
            mode='markers',
            name='Solution Found',
            marker=dict(color='#00FF00', size=15, symbol='star')
        )

    # Add reference line for perfect fitness
    fig.add_hline(
        y=28, 
        line_dash="dash", 
        line_color="rgba(0, 255, 0, 0.3)",
        annotation_text="Perfect Fitness (28)",
        annotation_position="right",
        annotation_font_color="white"
    )

    # Add annotations for initial and final fitness
    fig.add_annotation(
        x=0,
        y=fitness_history[0],
        text=f"Initial Fitness: {fitness_history[0]}",
        showarrow=True,
        arrowhead=1,
        ax=-50,
        ay=-30,
        font=dict(size=14, color='#00B5F7'),
        bgcolor='rgba(50, 50, 50, 0.8)'
    )

    if solution_point.empty:
        final_gen = len(fitness_history) - 1
        final_fitness = fitness_history[-1]
        annotation_text = f"Final Fitness: {final_fitness}"
    else:
        final_gen = solution_point['Generation'].iloc[0]
        annotation_text = "Solution Found!"

    fig.add_annotation(
        x=final_gen,
        y=fitness_history[final_gen],
        text=annotation_text,
        showarrow=True,
        arrowhead=1,
        ax=50,
        ay=-50,
        font=dict(size=14, color='#00FF00'),
        bgcolor='rgba(50, 50, 50, 0.8)'
    )

    # Final adjustments
    fig.update_layout(
        xaxis=dict(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False),
        yaxis=dict(gridcolor='rgba(128, 128, 128, 0.2)', zeroline=False, range=[min(fitness_history) - 1, 29]),
        height = 700,
        width=1500,
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01,
            bgcolor='rgba(50, 50, 50, 0.8)'
        ),
        hovermode='x unified',
        margin=dict(t=100, b=50, l=50, r=50)
    )

    print(f"Solution found in {final_gen + 1} generations")
    print(f"Final fitness: {fitness_history[final_gen]}")
    fig.show()

In [125]:
# Genetic Algorithm

class ExtendedGeneticAlgorithm:
    def __init__(self, board_size, population_size, max_generations, tournament_size, initial_mutation_rate):
        self.board_size = board_size
        self.population_size = population_size
        self.mutation_probability = initial_mutation_rate
        self.last_state = None
        self.previous_best_fitness = 0
        self.goal_fitness = int(((board_size ** 2) - board_size) / 2)
        self.inital_population = [EightQueensState(board_size) for _ in range(population_size)]
        self.current_population = self.inital_population
        self.max_generations = max_generations
        self.tournament_size = tournament_size


    # Extension
    def tournament_selection(self, population_data):
        # Tournament selection implementation
        tournament = random.sample(population_data, self.tournament_size)
        return max(tournament, key=lambda x: x['fitness'])['state']


    def routlette_wheel_selection(self, population_data):
        # Generate a random number between 0 and 1
        random_value = np.random.random()
        
        # Use cumulative probability to select an individual
        cumulative_probability = 0.0
        for individual in population_data:
            cumulative_probability += individual['selection_prob']
            if cumulative_probability > random_value:
                return individual['state']


    # Extension
    def adaptive_mutation(self, current_fitness):
        # Adaptive mutation rate based on fitness improvement
        if current_fitness < self.previous_best_fitness:
            self.mutation_probability *= 1.1  # Increase mutation rate if fitness decreases
        else:
            self.mutation_probability *= 0.9  # Decrease mutation rate if fitness improves
        # Keep mutation rate within reasonable bounds
        self.mutation_probability = min(0.4, max(0.01, self.mutation_probability))
        self.previous_best_fitness = current_fitness


    # Extension
    def improved_crossover(self, parent1, parent2):
        # Multi-point crossover
        points = sorted(random.sample(range(1, len(parent1)), 2))
        child1 = np.concatenate([
            parent1[:points[0]],
            parent2[points[0]:points[1]],
            parent1[points[1]:]
        ])
        child2 = np.concatenate([
            parent2[:points[0]],
            parent1[points[0]:points[1]],
            parent2[points[1]:]
        ])
        return child1, child2


    def crossover(self, parent1, parent2):
        # Perform a single-point crossover to produce two offspring via randomisation
        idx = np.random.randint(1, len(parent1))  

        # Apply the cross over of two parents to product two children 
        return np.concatenate([parent1[:idx], parent2[idx:]]), np.concatenate([parent2[:idx], parent1[idx:]])


    # Extension
    def calculate_diversity(self):
        # Calculate population diversity using Hamming distance
        diversity = 0
        for i in range(len(self.current_population)):
            for j in range(i+1, len(self.current_population)):
                diversity += np.sum(self.current_population[i].state != 
                                  self.current_population[j].state)
        return diversity / (len(self.current_population) * (len(self.current_population) - 1))
    

    def mutate(self, state):
        # Randomly mutate a queen's position in one column with a certain probability
        if np.random.random() < self.mutation_probability:

             # Select a random column to mutate
            idx = np.random.randint(len(state))  

            # Randomly choose a new row position for the queen in the selected column, excluding current row 
            state[idx] = np.random.choice(list(set(range(self.board_size)) - {state[idx]}))
        return state


    def create_next_generation(self, population_data):
        next_generation = []
        
        # Elitism - preserve best individuals
        # elite_size = max(2, self.population_size // 3)
        # elite = [EightQueensState(self.board_size, x['state']) for x in population_data[:elite_size]]
        # next_generation.extend(elite)

        # Add elitism - keep the best individual from the current generation
        # next_generation.extend([EightQueensState(self.board_size, individual['state']) for individual in population_data[:1]])

        # Calculate current diversity
        # diversity = self.calculate_diversity()
        
        # Fill the rest of the population
        while len(next_generation) < self.population_size:

            # parent1 = self.tournament_selection(population_data)
            # parent2 = self.tournament_selection(population_data)

            parent1 = self.routlette_wheel_selection(population_data)
            parent2 = self.routlette_wheel_selection(population_data)
            
            # child1, child2 = self.improved_crossover(parent1, parent2)
            child1, child2 = self.crossover(parent1, parent2)
            
            # # Adaptive mutation based on population diversity
            # if diversity < 0:  # Increase mutation if diversity is low
                # self.mutation_probability = min(0.3, self.mutation_probability * 1.5)
            
            child1 = self.mutate(child1)
            child2 = self.mutate(child2)
            
            next_generation.extend([
                EightQueensState(self.board_size, child1),
                EightQueensState(self.board_size, child2)
            ])
        
        self.current_population = next_generation[:self.population_size]


    def calculate_population_fitness(self):
        # Calculate fitness and selection probability for each individual in the population
        population_data = []
        total_fitness = 0
        
        for individual in self.current_population:
            fitness = individual.fitness()
            population_data.append({
                'state': individual.state,
                'fitness': fitness
            })
            total_fitness += fitness
        
        # Calculate selection probability for each individual based on fitness
        for data in population_data:
            data['selection_prob'] = data['fitness'] / total_fitness
            
        # Return the sorted the population by the highest selection probabilities
        return sorted(population_data, key=lambda x: x['selection_prob'], reverse=True)


    def run(self):
        generation = 0
        fitness_history = []
        diversity_history = []
        
        while generation < self.max_generations:
            population_data = self.calculate_population_fitness()
            # current_diversity = self.calculate_diversity()
            
            # Store history
            best_fitness = population_data[0]['fitness']
            fitness_history.append(best_fitness)
            # diversity_history.append(current_diversity)

            # if generation > 100:
            #     idx = len(fitness_history) - 1
            #     if best_fitness == fitness_history[idx - 10]:
            #         self.current_population = self.inital_population
            #         # generation = 0
            
            # Adapt mutation rate
            self.adaptive_mutation(best_fitness)
            
            if best_fitness == self.goal_fitness:
                self.last_state = population_data[0]['state']
                # print(f"Solution found in generation {generation}")
                print(f"Final mutation rate: {self.mutation_probability:.3f}")
                # print(f"Final diversity: {current_diversity:.3f}")
                return fitness_history, diversity_history, generation
            
            self.create_next_generation(population_data)
            generation += 1
        
        return fitness_history, diversity_history, generation
    

    def display_solution(self):
        # Display the chessboard with queens in the positions defined by self.last_state
        for row in range(self.board_size):
            print(f'{self.board_size - row} |', end=' ')
            for col in range(self.board_size):
                if self.last_state[col] == row:
                    print('Q ', end='')  
                else:
                    print('- ', end='') 
            print(f'|')

In [126]:
# Run

def run_multiple_trials(trials):
    all_times = []
    all_generations = []
    successes = 0
    max_generations = 1000  # Define a max generation limit to consider a run successful

    fitness_histories = []
    diversity_histories = []

    for trial in range(trials):
        # print(f"\nTrial {trial + 1}:")
        
        start_time = time.time()
        
        ga = ExtendedGeneticAlgorithm(
            board_size=8,
            population_size=100,
            initial_mutation_rate=0.1,
            tournament_size = 5,
            max_generations=1000
        )
        
        fitness_history, diversity_history, generations = ga.run()  # Assume `run()` returns (fitness_history, generations)

        fitness_histories.append(fitness_history)
        diversity_histories.append(diversity_history)
        
        end_time = time.time()
        time_taken = end_time - start_time
        all_times.append(time_taken)
        all_generations.append(generations)
        
        if generations < max_generations:
            successes += 1  # Count as success if solution found within max_generations
        
        print(f"Solution found in: {time_taken:.2f} seconds, Generations: {generations}")

    # Calculate and print metrics
    avg_time = np.mean(all_times)
    std_time = np.std(all_times)
    avg_generations = np.mean(all_generations)
    std_generations = np.std(all_generations)
    cv_generations = std_generations / avg_generations if avg_generations > 0 else float('inf')
    success_rate = successes / trials * 100
    max_time = np.max(all_times)
    min_time = np.min(all_times)

    # print("\n--- Performance Metrics ---")
    print(f"Average Time: {avg_time:.2f} seconds")
    print(f"Standard Deviation of Time: {std_time:.2f} seconds")
    print(f"Max Time: {max_time:.2f} seconds")
    print(f"Min Time: {min_time:.2f} seconds")
    # print(f"Average Generations to Solution: {avg_generations:.2f}")
    # print(f"Standard Deviation of Generations: {std_generations:.2f}")
    # print(f"Coefficient of Variation of Generations: {cv_generations:.2f}")
    # print(f"Success Rate: {success_rate:.2f}%")

    return fitness_histories, diversity_histories

# Run the multiple trials
fitness_histories = run_multiple_trials(10)

KeyboardInterrupt: 

In [102]:
for fitness_history in fitness_histories:
    plot_ga(fitness_history)

Solution found in 106 generations
Final fitness: 28


Solution found in 1000 generations
Final fitness: 27


Solution found in 508 generations
Final fitness: 28


Solution found in 249 generations
Final fitness: 28


Solution found in 312 generations
Final fitness: 28


Solution found in 127 generations
Final fitness: 28


Solution found in 222 generations
Final fitness: 28


Solution found in 78 generations
Final fitness: 28


Solution found in 1000 generations
Final fitness: 27


Solution found in 344 generations
Final fitness: 28
